[Reference](https://levelup.gitconnected.com/how-to-build-a-high-performance-free-elt-pipeline-locally-using-duckdb-8fe9d7235079)

```
pip install duckdb pandas
```


In [1]:
import duckdb
import os
import json

# Define constants for file paths
DB_PATH = 'urbancycle_warehouse.duckdb'
ORDERS_CSV = 'orders_data.csv'
PRODUCTS_JSON = 'product_inventory.json'

# Assume the two data sources are ORDERS_CSV and PRODUCTS_JSON
# Below are what they look like


# 1. Orders Data (CSV)
csv_content = (
        "order_id,product_sku,quantity,sale_date,customer_id\n"
        "1001,TIRE_R_001,2,2023-10-01,C123\n"
        "1002,HELMET_A_005,1,2023-10-01,C124\n"
        "1003,TIRE_R_001,5,2023-10-02,C125\n"
)

# 2. Products Data (JSON - Simulating API dump)
products_data = [
        {'sku': 'TIRE_R_001', 'name': 'Road Tire Pro', 'category': 'Wheels', 'unit_price': 50.00, 'is_active': True},
        {'sku': 'HELMET_A_005', 'name': 'Aero Helmet 5', 'category': 'Accessories', 'unit_price': 120.00, 'is_active': True}
]

def run_el_phase():
    """Connects to DuckDB and loads raw data from CSV and JSON."""

    print("\n--- Starting EL Phase (Extract & Load) ---")

    # Use 'with' for safe connection management
    try:
        with duckdb.connect(DB_PATH) as con:
            # Create a dedicated schema for raw, untransformed data
            con.execute("CREATE SCHEMA IF NOT EXISTS raw_layer")

            # 1. Load CSV Orders Data (Native Ingest)
            print(f"Loading {ORDERS_CSV} into raw_layer.orders...")
            con.execute(f"""
                CREATE OR REPLACE TABLE raw_layer.orders AS
                SELECT * FROM read_csv_auto('{ORDERS_CSV}')
            """)

            # 2. Load JSON Products Data (Native JSON Read)
            # We treat the JSON file as a source table.
            # The 'UNNEST' function is key to flattening complex arrays in JSON.
            print(f"Loading {PRODUCTS_JSON} into raw_layer.products...")
            con.execute(f"""
                CREATE OR REPLACE TABLE raw_layer.products AS
                SELECT
                    unnest(json_array_of_objects('{PRODUCTS_JSON}')) as product_data
            """)

            # Now, flatten the complex JSON structure into a usable table
            con.execute("""
                CREATE OR REPLACE TABLE raw_layer.products_flat AS
                SELECT
                    product_data ->> 'sku' AS product_sku,
                    product_data ->> 'name' AS product_name,
                    product_data ->> 'category' AS product_category,
                    (product_data ->> 'unit_price')::FLOAT AS unit_price,
                    (product_data ->> 'is_active')::BOOLEAN AS is_active
                FROM raw_layer.products
            """)

            # Verify the load count
            order_count = con.sql("SELECT count(order_id) FROM raw_layer.orders").fetchone()[0]
            product_count = con.sql("SELECT count(product_sku) FROM raw_layer.products_flat").fetchone()[0]

            print(f"✅ Loaded {order_count} orders and {product_count} products.")
            return True

    except duckdb.Error as e:
        print(f"❌ DuckDB Error during EL phase: {e}")
        return False
    except IOError as e:
        print(f"❌ File I/O Error: Ensure {ORDERS_CSV} and {PRODUCTS_JSON} exist. {e}")
        return False

def run_t_phase():
    """Performs transformation and aggregation for the unified report."""

    print("\n--- Starting T Phase (Transform) ---")

    try:
        with duckdb.connect(DB_PATH) as con:
            con.execute("CREATE SCHEMA IF NOT EXISTS analytical_layer")

            # The transformation query for the report sales by product and category
            transformation_query = """
                -- Create the final report table for the business users
                CREATE OR REPLACE TABLE analytical_layer.revenue_by_category AS
                SELECT
                    p.product_category,
                    p.product_name,
                    CAST(STRFTIME(o.sale_date, '%Y-%m') AS VARCHAR) AS sales_month,
                    SUM(o.quantity) AS total_units_sold,
                    SUM(o.quantity * p.unit_price) AS total_revenue_usd,
                    COUNT(DISTINCT o.customer_id) AS distinct_customers
                FROM raw_layer.orders o
                -- Key join operation
                JOIN raw_layer.products_flat p ON o.product_sku = p.product_sku
                WHERE p.is_active = TRUE -- Only analyze active products
                GROUP BY 1, 2, 3
                ORDER BY total_revenue_usd DESC;
            """

            con.execute(transformation_query)
            print("✅ Transformation query executed successfully.")

            # Display the final report
            print("\n--- Final UrbanCycle Revenue Report ---")
            result_df = con.sql("SELECT * FROM analytical_layer.revenue_by_category").df()
            print(result_df.to_markdown(index=False))

            return True

    except duckdb.Error as e:
        print(f"❌ DuckDB Error during T phase: {e}. Check your SQL syntax or schema names.")
        return False

In [2]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime
from functools import partial

# We use functools.partial to pass the connection functions to the operator
# The functions run_el_phase and run_t_phase defined above are used here.

def pipeline_start():
    print("Pipeline started.")

def pipeline_end():
    print("Pipeline finished. The report is ready in the DuckDB file.")

with DAG(
    dag_id='urbancycle_duckdb_elt',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@daily',
    catchup=False,
    tags=['duckdb', 'elt', 'revenue_report']
) as dag:

    start_task = PythonOperator(
        task_id='start_pipeline',
        python_callable=pipeline_start
    )

    # Task 1: Extract and Load
    el_task = PythonOperator(
        task_id='extract_and_load',
        python_callable=run_el_phase,
        # Airflow will mark the task failed if the function returns False
        do_xcom_push=False
    )

    # Task 2: Transform
    t_task = PythonOperator(
        task_id='transform_and_report',
        python_callable=run_t_phase,
        do_xcom_push=False
    )

    end_task = PythonOperator(
        task_id='end_pipeline',
        python_callable=pipeline_end
    )

    # Define the sequence of tasks
    start_task >> el_task >> t_task >> end_task